In [1]:
#from fastapi import FastAPI, UploadFile, File, Form
import os
import json
import promptManager
import outputFormating
import pandas as pd


In [2]:

#Get marketing material
marketing_material = input("Enter marketing material: ") 

# Select product - as you add more products it would be better to start using IDs but we are using names for ease of reading here.
product=None
while product is None:
    product_num=input("1: Clareon® Toric Intraocular Lens (IOL) or 2:  PRECISION7® Contact Lenses")

    if product_num=="1":
        product="ClareonPanOptix"

    elif product_num=="2":
        product="TOTAL30"
    else:
        print("Invalid input, please enter 1 or 2.")
    
    print("Selected product: ", product)

Selected product:  TOTAL30


In [3]:
prompt=promptManager.buildPrompt(marketing_material, product)
print("Prompt built: ", prompt)

#Get response from LLM
output=promptManager.getResponseFromLLM(prompt)



Prompt built:  You are a compliance grader evaluating medical marketing content in the U.S.

Your goal is to:
1) Reward similarity with approved claims.
2) Penalize conflicts with FDA + FTC requirements.
3) Return structured JSON referencing policy IDs.

---

## Marketing Text
good for eyes

---

## Policy Groups

Approved Claims (positive alignment)
- (approved:0) TOTAL30® contact lenses that feel like nothing, even at day 30.
- (approved:1) The first and only monthly replacement Water Gradient contact lenses.
- (approved:2) TOTAL30® contact lenses feature a gradual transition in water content, from 55 % at the core to nearly 100 % water at the outermost surface.
- (approved:3) Water Gradient Technology in TOTAL30 contact lenses lasts for a full 30 days.
- (approved:4) CELLIGENT® Technology creates a dynamic lens surface that biomimics the corneal surface.
- (approved:5) Helps resist the adherence of bacteria and lipids for a clean lens.
- (approved:6) Water Gradient delivers superior

In [4]:
print("Output from LLM: ", output)

Output from LLM:  {'evaluations': [{'policy_id': 'approved:0', 'policy': 'TOTAL30® contact lenses that feel like nothing, even at day\u202f30.', 'type': 'approved', 'grade': 20, 'reason': "Text 'good for eyes' is too vague and does not convey same meaning or detail."}, {'policy_id': 'approved:1', 'policy': 'The first and only monthly replacement Water Gradient contact lenses.', 'type': 'approved', 'grade': 20, 'reason': 'No mention of product uniqueness or monthly replacement in text.'}, {'policy_id': 'approved:2', 'policy': 'TOTAL30® contact lenses feature a gradual transition in water content, from 55\u202f% at the core to nearly 100\u202f% water at the outermost surface.', 'type': 'approved', 'grade': 20, 'reason': 'Text does not reference technical lens features or water content.'}, {'policy_id': 'approved:3', 'policy': 'Water Gradient Technology in TOTAL30 contact lenses lasts for a full 30 days.', 'type': 'approved', 'grade': 20, 'reason': 'No connection to technology durability 

In [6]:

df = pd.DataFrame(output['evaluations'])


In [8]:
df.head(60)

,policy_id,policy,type,grade,reason
0,approved:0,TOTAL30® contact lenses that feel like nothing...,approved,20,Text 'good for eyes' is too vague and does not...
1,approved:1,The first and only monthly replacement Water G...,approved,20,No mention of product uniqueness or monthly re...
2,approved:2,TOTAL30® contact lenses feature a gradual tran...,approved,20,Text does not reference technical lens feature...
3,approved:3,Water Gradient Technology in TOTAL30 contact l...,approved,20,No connection to technology durability or dura...
4,approved:4,CELLIGENT® Technology creates a dynamic lens s...,approved,20,Text does not reference CELLIGENT technology o...
5,approved:5,Helps resist the adherence of bacteria and lip...,approved,20,Text does not claim bacterial or lipid resista...
6,approved:6,Water Gradient delivers superior softness and ...,approved,20,Text does not compare softness or lubricity to...
7,approved:7,Class 1 UV Blocking delivers the highest level...,approved,20,No mention of UV blocking properties.
8,approved:8,The first and only monthly replacement Water G...,approved,20,No mention of toric variant or uniqueness.
9,approved:9,"TOTAL30 delivers the only Water Gradient, reus...",approved,20,The text 'good for eyes' is too generic and do...


In [ ]:
#get catagory scores
approved_scores = df[df['type'] == 'approved']['grade']
fda_scores = df[df['type'] == 'fda']['grade']
ftc_scores = df[df['type'] == 'ftc']['grade']

approved_score = approved_scores.max() if not approved_scores.empty else None

def compute_category_score(df, category, threshold=70):
    scores = df[df['type'] == category]['grade']

    if scores.empty:
        return 0
    
    base = scores.mean()
    non_compliant_ratio = (scores < threshold).mean()
    penalty = non_compliant_ratio * 100
    
    final = base - penalty
    return max(0, min(100, final))  # clamp

fda_score = compute_category_score(df, "fda")
ftc_score = compute_category_score(df, "ftc")

#Overall score is the average of the category scores
category_scores = [score for score in [approved_score, fda_score, ftc_score] if score is not None] 
